| Model                                                    | Company / Organization                       |
| -------------------------------------------------------- | -------------------------------------------- |
| GPT-Image-1                                              | OpenAI                                       |
| Imagen 4 Ultra / Imagen 4                                | Google / DeepMind (via Gemini / Vertex AI)   |
| Imagen 4 Generate Preview                                | Google (same Imagen / Gemini infrastructure) |
| Imagen 3 Generate 002                                    | Google / DeepMind                            |
| FLUX.1 Kontext Max / FLUX.1 Kontext Pro / FLUX.1 Kontext | Black Forest Labs (BFL)                      |
| FLUX.1 [pro] Ultra (if listed)                           | Black Forest Labs                            |
| Recraft V3                                               | Recraft, Inc.                                |
| Ideogram 3.0                                             | Ideogram AI                                  |
| Qwen Image                                               | Alibaba / Qwen team                          |
| Gemini 2.0 Flash Image Gen (Preview) / Gemini line       | Google / DeepMind                            |
| Seedream 4.0                                             | ByteDance                                    |
| Amazon Nova Premier                                      | AWS / Amazon                                 |


In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai as genai
from google.cloud import aiplatform

# Load environment variables from .env file
load_dotenv()

In [ ]:
# Load environment keys
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
BFL_API_KEY = os.getenv('BLACK_FOREST_LABS_API_KEY')  # Black Forest Labs
RECRAFT_API_KEY = os.getenv('RECRAFT_API_KEY')
IDEOGRAM_API_KEY = os.getenv('IDEOGRAM_API_KEY')
QWEN_API_KEY = os.getenv('OPENROUTER_API_KEY')
FAL_API_KEY= os.getenv('FAL_API_KEY')

In [ ]:
# Load LeWitt instructions from CSV
lewitt_df = pd.read_csv('lewitt_instructions.csv')

print(f"Loaded {len(lewitt_df)} LeWitt instructions")
print(f"\nColumns: {list(lewitt_df.columns)}")
print(f"\nFirst few instructions:")
print(lewitt_df.head())

In [ ]:
# Imports and helper functions
import requests
import base64
from datetime import datetime
from pathlib import Path
import time
import json
from openai import OpenAI
import google.generativeai as genai

# Create base directory for storing images
BASE_DIR = Path.cwd()
DATE_SUFFIX = datetime.now().strftime("%Y%m%d")

def create_output_dir(provider_name):
    """Create output directory for a provider"""
    dir_path = BASE_DIR / f"{provider_name}_{DATE_SUFFIX}"
    dir_path.mkdir(exist_ok=True)
    return dir_path

def save_image(image_data, output_path):
    """Save image data to file"""
    with open(output_path, 'wb') as f:
        f.write(image_data)
    print(f"✓ Saved: {output_path}")

print("✓ Helper functions loaded")

In [ ]:
# 1. OpenAI GPT-Image-1 (DALL-E 3)
def generate_openai_image(prompt, instruction_id, api_key):
    """Generate image using OpenAI's DALL-E"""
    if not api_key:
        print("⚠ OpenAI API key not found")
        return None
    
    client = OpenAI(api_key=api_key)
    output_dir = create_output_dir("openai")
    
    try:
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )
        
        # Download the image
        image_url = response.data[0].url
        image_response = requests.get(image_url)
        
        if image_response.status_code == 200:
            output_path = output_dir / f"{instruction_id}.jpg"
            save_image(image_response.content, output_path)
            return output_path
    except Exception as e:
        print(f"✗ OpenAI Error: {e}")
        return None

print("✓ OpenAI function defined")


In [ ]:

from google import genai
from google.genai import types
from io import BytesIO

client = genai.Client()

# 2a. Google Imagen 4 Ultra
def generate_google_imagen_4_ultra(prompt, instruction_id, api_key):
    """Generate image using Google's Imagen 4 Ultra"""
    if not api_key:
        print("⚠ Google API key not found")
        return None
    
    output_dir = create_output_dir("google_imagen_4_ultra")
    
    
    try:
        import google.genai.types as types
        
        # Create client with API key
        client = genai.Client(api_key=api_key)
        
        # Generate images
        response = client.models.generate_images(
            model='imagen-4.0-ultra-generate-001',
            prompt=prompt,
            config=types.GenerateImagesConfig(
                number_of_images=1,
            )
        )
        
        # Save generated images
        saved_paths = []
        for idx, generated_image in enumerate(response.generated_images):
            filename = f"{instruction_id}_{idx+1}.png"
            filepath = output_dir / filename
            generated_image.image.save(filepath)
            saved_paths.append(filepath)
            print(f"✓ Saved: {filepath}")
        
        return saved_paths[0] if saved_paths else None
        
    except Exception as e:
        print(f"✗ Google Imagen 4 Ultra Error: {e}")
        return None

In [ ]:
# 2b. Google Imagen 4
from google import genai
from google.genai import types
from io import BytesIO

client = genai.Client()


def generate_google_imagen_4(prompt, instruction_id, api_key):
    """Generate image using Google's Imagen 4"""
    if not api_key:
        print("⚠ Google API key not found")
        return None
    
    output_dir = create_output_dir("google_imagen_4")
    
    try:
        import google.genai.types as types
        
        # Create client with API key
        client = genai.Client(api_key=api_key)
        
        # Generate images
        response = client.models.generate_images(
            model='imagen-4.0-generate-001',
            prompt=prompt,
            config=types.GenerateImagesConfig(
                number_of_images=1,
            )
        )
        
        # Save generated images
        saved_paths = []
        for idx, generated_image in enumerate(response.generated_images):
            filename = f"{instruction_id}_{idx+1}.png"
            filepath = output_dir / filename
            generated_image.image.save(filepath)
            saved_paths.append(filepath)
            print(f"✓ Saved: {filepath}")
        
        return saved_paths[0] if saved_paths else None
        
    except Exception as e:
        print(f"✗ Google Imagen 4 Error: {e}")
        return None

print("✓ Google Imagen 4 function defined")


In [ ]:
# 2c. Google Imagen 3
from google import genai
from google.genai import types
from io import BytesIO

client = genai.Client()


def generate_google_imagen_3(prompt, instruction_id, api_key):
    """Generate image using Google's Imagen 3"""
    if not api_key:
        print("⚠ Google API key not found")
        return None
    
    output_dir = create_output_dir("google_imagen_3")
    
    try:
        import google.genai.types as types
        
        # Create client with API key
        client = genai.Client(api_key=api_key)
        
        # Generate images
        response = client.models.generate_images(
            model='imagen-3.0-generate-002',
            prompt=prompt,
            config=types.GenerateImagesConfig(
                number_of_images=1,
            )
        )
        
        # Save generated images
        saved_paths = []
        for idx, generated_image in enumerate(response.generated_images):
            filename = f"{instruction_id}_{idx+1}.png"
            filepath = output_dir / filename
            generated_image.image.save(filepath)
            saved_paths.append(filepath)
            print(f"✓ Saved: {filepath}")
        
        return saved_paths[0] if saved_paths else None
        
    except Exception as e:
        print(f"✗ Google Imagen 3 Error: {e}")
        return None

print("✓ Google Imagen 3 function defined")


In [ ]:
# 3a. Black Forest Labs FLUX Pro 1.1
def generate_flux_pro_image(prompt, instruction_id, api_key):
    """Generate image using FLUX Pro 1.1"""
    if not api_key:
        print("⚠ BFL API key not found")
        return None
    
    output_dir = create_output_dir("flux_pro_1.1")
    
    try:
        headers = {
            "x-key": api_key,
            "Content-Type": "application/json"
        }
        
        payload = {
            "prompt": prompt,
            "width": 1024,
            "height": 1024,
        }
        
        # Submit request to BFL API
        response = requests.post(
            "https://api.bfl.ai/v1/flux-pro-1.1",
            headers=headers,
            json=payload
        )
        
        if response.status_code == 200:
            result = response.json()
            request_id = result.get('id')
            
            # Poll for result using the request ID
            max_attempts = 60
            for attempt in range(max_attempts):
                time.sleep(2)
                
                poll_response = requests.get(
                    f"https://api.bfl.ai/v1/get_result?id={request_id}",
                    headers={"x-key": api_key}
                )
                
                if poll_response.status_code == 200:
                    poll_result = poll_response.json()
                    status = poll_result.get('status')
                    
                    if status == 'Ready':
                        image_url = poll_result.get('result', {}).get('sample')
                        if image_url:
                            image_response = requests.get(image_url)
                            output_path = output_dir / f"{instruction_id}.jpg"
                            save_image(image_response.content, output_path)
                            return output_path
                    elif status in ['Error', 'Request Moderated']:
                        print(f"✗ FLUX Pro 1.1 Error: {poll_result.get('status')}")
                        return None
            
            print(f"✗ FLUX Pro 1.1 Error: Timeout waiting for result")
            return None
        else:
            print(f"✗ FLUX Pro 1.1 Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"✗ FLUX Pro 1.1 Error: {e}")
        return None

print("✓ FLUX Pro 1.1 function defined")


In [ ]:
# 3b. Black Forest Labs FLUX Pro 1.1 Ultra
def generate_flux_pro_ultra_image(prompt, instruction_id, api_key):
    """Generate image using FLUX Pro 1.1 Ultra"""
    if not api_key:
        print("⚠ BFL API key not found")
        return None
    
    output_dir = create_output_dir("flux_pro_1.1_ultra")
    
    try:
        headers = {
            "x-key": api_key,
            "Content-Type": "application/json"
        }
        
        payload = {
            "prompt": prompt,
            "width": 1024,
            "height": 1024,
            "safety_tolerance": 2,
        }
        
        # Submit request to BFL API
        response = requests.post(
            "https://api.bfl.ai/v1/flux-pro-1.1-ultra",
            headers=headers,
            json=payload
        )
        
        if response.status_code == 200:
            result = response.json()
            request_id = result.get('id')
            
            # Poll for result using the request ID
            max_attempts = 60
            for attempt in range(max_attempts):
                time.sleep(2)
                
                poll_response = requests.get(
                    f"https://api.bfl.ai/v1/get_result?id={request_id}",
                    headers={"x-key": api_key}
                )
                
                if poll_response.status_code == 200:
                    poll_result = poll_response.json()
                    status = poll_result.get('status')
                    
                    if status == 'Ready':
                        image_url = poll_result.get('result', {}).get('sample')
                        if image_url:
                            image_response = requests.get(image_url)
                            output_path = output_dir / f"{instruction_id}.jpg"
                            save_image(image_response.content, output_path)
                            return output_path
                    elif status in ['Error', 'Request Moderated']:
                        print(f"✗ FLUX Pro 1.1 Ultra Error: {poll_result.get('status')}")
                        return None
            
            print(f"✗ FLUX Pro 1.1 Ultra Error: Timeout waiting for result")
            return None
        else:
            print(f"✗ FLUX Pro 1.1 Ultra Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"✗ FLUX Pro 1.1 Ultra Error: {e}")
        return None

print("✓ FLUX Pro 1.1 Ultra function defined")


In [ ]:
# 3c. Black Forest Labs FLUX Kontext Pro
def generate_flux_kontext_pro_image(prompt, instruction_id, api_key):
    """Generate image using FLUX Kontext Pro"""
    if not api_key:
        print("⚠ BFL API key not found")
        return None
    
    output_dir = create_output_dir("flux_kontext_pro")
    
    try:
        headers = {
            "x-key": api_key,
            "Content-Type": "application/json"
        }
        
        payload = {
            "prompt": prompt,
            "width": 1024,
            "height": 1024,
        }
        
        # Submit request to BFL API
        response = requests.post(
            "https://api.bfl.ai/v1/flux-kontext-pro",
            headers=headers,
            json=payload
        )
        
        if response.status_code == 200:
            result = response.json()
            request_id = result.get('id')
            
            # Poll for result using the request ID
            max_attempts = 60
            for attempt in range(max_attempts):
                time.sleep(2)
                
                poll_response = requests.get(
                    f"https://api.bfl.ai/v1/get_result?id={request_id}",
                    headers={"x-key": api_key}
                )
                
                if poll_response.status_code == 200:
                    poll_result = poll_response.json()
                    status = poll_result.get('status')
                    
                    if status == 'Ready':
                        image_url = poll_result.get('result', {}).get('sample')
                        if image_url:
                            image_response = requests.get(image_url)
                            output_path = output_dir / f"{instruction_id}.jpg"
                            save_image(image_response.content, output_path)
                            return output_path
                    elif status in ['Error', 'Request Moderated']:
                        print(f"✗ FLUX Kontext Pro Error: {poll_result.get('status')}")
                        return None
            
            print(f"✗ FLUX Kontext Pro Error: Timeout waiting for result")
            return None
        else:
            print(f"✗ FLUX Kontext Pro Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"✗ FLUX Kontext Pro Error: {e}")
        return None

print("✓ FLUX Kontext Pro function defined")


In [ ]:
# 3d. Black Forest Labs FLUX Kontext Max
def generate_flux_kontext_max_image(prompt, instruction_id, api_key):
    """Generate image using FLUX Kontext Max"""
    if not api_key:
        print("⚠ BFL API key not found")
        return None
    
    output_dir = create_output_dir("flux_kontext_max")
    
    try:
        headers = {
            "x-key": api_key,
            "Content-Type": "application/json"
        }
        
        payload = {
            "prompt": prompt,
            "width": 1024,
            "height": 1024,
        }
        
        # Submit request to BFL API
        response = requests.post(
            "https://api.bfl.ai/v1/flux-kontext-max",
            headers=headers,
            json=payload
        )
        
        if response.status_code == 200:
            result = response.json()
            request_id = result.get('id')
            
            # Poll for result using the request ID
            max_attempts = 60
            for attempt in range(max_attempts):
                time.sleep(2)
                
                poll_response = requests.get(
                    f"https://api.bfl.ai/v1/get_result?id={request_id}",
                    headers={"x-key": api_key}
                )
                
                if poll_response.status_code == 200:
                    poll_result = poll_response.json()
                    status = poll_result.get('status')
                    
                    if status == 'Ready':
                        image_url = poll_result.get('result', {}).get('sample')
                        if image_url:
                            image_response = requests.get(image_url)
                            output_path = output_dir / f"{instruction_id}.jpg"
                            save_image(image_response.content, output_path)
                            return output_path
                    elif status in ['Error', 'Request Moderated']:
                        print(f"✗ FLUX Kontext Max Error: {poll_result.get('status')}")
                        return None
            
            print(f"✗ FLUX Kontext Max Error: Timeout waiting for result")
            return None
        else:
            print(f"✗ FLUX Kontext Max Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"✗ FLUX Kontext Max Error: {e}")
        return None

print("✓ FLUX Kontext Max function defined")


In [ ]:
# 4. Recraft V3
def generate_recraft_image(prompt, instruction_id, api_key):
    """Generate image using Recraft V3"""
    if not api_key:
        print("⚠ Recraft API key not found")
        return None
    
    output_dir = create_output_dir("recraft")
    
    try:
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        
        payload = {
            "prompt": prompt,
            "model": "recraftv3",
            "size": "1024x1024",
        }
        
        # Using official Recraft API endpoint
        response = requests.post(
            "https://external.api.recraft.ai/v1/images/generations",
            headers=headers,
            json=payload
        )
        
        if response.status_code == 200:
            result = response.json()
            # Recraft V3 returns base64 encoded image
            image_data = result.get('data', [{}])[0].get('url')
            if image_data:
                # If URL is provided
                if image_data.startswith('http'):
                    image_response = requests.get(image_data)
                    output_path = output_dir / f"{instruction_id}.jpg"
                    save_image(image_response.content, output_path)
                    return output_path
                # If base64 is provided
                elif image_data.startswith('data:image'):
                    base64_data = image_data.split(',')[1]
                    image_bytes = base64.b64decode(base64_data)
                    output_path = output_dir / f"{instruction_id}.jpg"
                    save_image(image_bytes, output_path)
                    return output_path
        else:
            print(f"✗ Recraft Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"✗ Recraft Error: {e}")
        return None

print("✓ Recraft function defined")


In [ ]:
# 5. Ideogram
def generate_ideogram_image(prompt, instruction_id, api_key):
    """Generate image using Ideogram V3"""
    if not api_key:
        print("⚠ Ideogram API key not found")
        return None
    
    output_dir = create_output_dir("ideogram")
    
    try:
        # Generate with Ideogram 3.0
        response = requests.post(
            "https://api.ideogram.ai/v1/ideogram-v3/generate",
            headers={
                "Api-Key": api_key
            },
            json={
                "prompt": prompt,
                "rendering_speed": "TURBO"
            }
        )
        
        if response.status_code == 200:
            result = response.json()
            # Get image URL from response
            image_url = result['data'][0]['url']
            
            # Download and save the image
            image_response = requests.get(image_url)
            output_path = output_dir / f"{instruction_id}.png"
            save_image(image_response.content, output_path)
            return output_path
        else:
            print(f"✗ Ideogram Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"✗ Ideogram Error: {e}")
        return None

print("✓ Ideogram function defined")



In [ ]:
# 7. FAL (Flux on FAL)
def generate_fal_image(prompt, instruction_id, api_key):
    """Generate image using FAL"""
    if not api_key:
        print("⚠ FAL API key not found")
        return None
    
    output_dir = create_output_dir("qwen")
    
    try:
        headers = {
            "Authorization": f"Key {api_key}",
            "Content-Type": "application/json"
        }
        
        payload = {
            "prompt": prompt,
            "image_size": "square_hd",
            "num_inference_steps": 28,
            "num_images": 1
        }
        
        response = requests.post(
            "https://fal.run/fal-ai/qwen-image",
            headers=headers,
            json=payload
        )
        
        if response.status_code == 200:
            result = response.json()
            images = result.get('images', [])
            if images:
                image_url = images[0].get('url')
                if image_url:
                    image_response = requests.get(image_url)
                    output_path = output_dir / f"{instruction_id}.jpg"
                    save_image(image_response.content, output_path)
                    return output_path
        else:
            print(f"✗ FAL Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"✗ FAL Error: {e}")
        return None

print("✓ FAL function defined")


In [ ]:
# Test with first row of lewitt_df
test_row = lewitt_df.iloc[0]
test_id = test_row['id']
test_instruction = test_row['instruction']

print(f"Testing with LeWitt Wall Drawing #{test_id}")
print(f"Instruction: {test_instruction}")
print(f"\nGenerating images with available APIs...")
print("=" * 70)

# Test each provider
results = {}

# 1. OpenAI
print("\n1. Testing OpenAI DALL-E 3...")
result = generate_openai_image(test_instruction, test_id, OPENAI_API_KEY)
results['openai'] = result

# 2. Google Imagen variants
print("\n2a. Testing Google Imagen 4 Ultra...")
result = generate_google_imagen_4_ultra(test_instruction, test_id, GOOGLE_API_KEY)
results['google_imagen_4_ultra'] = result

print("\n2b. Testing Google Imagen 4...")
result = generate_google_imagen_4(test_instruction, test_id, GOOGLE_API_KEY)
results['google_imagen_4'] = result

print("\n2c. Testing Google Imagen 3...")
result = generate_google_imagen_3(test_instruction, test_id, GOOGLE_API_KEY)
results['google_imagen_3'] = result

# 3. Black Forest Labs FLUX variants
print("\n3a. Testing FLUX Pro 1.1...")
result = generate_flux_pro_image(test_instruction, test_id, BFL_API_KEY)
results['flux_pro_1.1'] = result

print("\n3b. Testing FLUX Pro 1.1 Ultra...")
result = generate_flux_pro_ultra_image(test_instruction, test_id, BFL_API_KEY)
results['flux_pro_1.1_ultra'] = result

print("\n3c. Testing FLUX Kontext Pro...")
result = generate_flux_kontext_pro_image(test_instruction, test_id, BFL_API_KEY)
results['flux_kontext_pro'] = result

print("\n3d. Testing FLUX Kontext Max...")
result = generate_flux_kontext_max_image(test_instruction, test_id, BFL_API_KEY)
results['flux_kontext_max'] = result

# 4. Recraft
print("\n4. Testing Recraft V3...")
result = generate_recraft_image(test_instruction, test_id, RECRAFT_API_KEY)
results['recraft'] = result

# 5. Ideogram
print("\n5. Testing Ideogram...")
result = generate_ideogram_image(test_instruction, test_id, IDEOGRAM_API_KEY)
results['ideogram'] = result

# 7. FAL
print("\n7. Testing FAL...")
result = generate_fal_image(test_instruction, test_id, FAL_API_KEY)
results['fal'] = result

# Summary
print("\n" + "=" * 70)
print("SUMMARY:")
print("=" * 70)
successful = [k for k, v in results.items() if v is not None]
failed = [k for k, v in results.items() if v is None]

print(f"✓ Successful: {len(successful)}/{len(results)}")
if successful:
    for provider in successful:
        print(f"  - {provider}: {results[provider]}")

print(f"\n✗ Failed: {len(failed)}/{len(results)}")
if failed:
    for provider in failed:
        print(f"  - {provider}")


In [ ]:
# Batch processing function for all instructions
def process_all_instructions(df, providers_config):
    """
    Process all LeWitt instructions with specified providers
    
    Args:
        df: DataFrame with LeWitt instructions
        providers_config: Dict mapping provider names to (function, api_key) tuples
    """
    total = len(df)
    results_log = []
    
    print(f"Processing {total} LeWitt instructions...")
    print("=" * 70)
    
    for idx, row in df.iterrows():
        instruction_id = row['id']
        instruction_text = row['instruction']
        
        print(f"\n[{idx + 1}/{total}] Processing Wall Drawing #{instruction_id}")
        print(f"Instruction: {instruction_text[:80]}...")
        
        row_results = {'id': instruction_id, 'instruction': instruction_text}
        
        for provider_name, (func, api_key) in providers_config.items():
            try:
                print(f"  → {provider_name}...", end=" ")
                result = func(instruction_text, instruction_id, api_key)
                row_results[provider_name] = str(result) if result else "Failed"
                
                if result:
                    print("✓")
                else:
                    print("✗")
                    
                # Rate limiting - sleep between requests
                time.sleep(1)
            except Exception as e:
                print(f"✗ Error: {e}")
                row_results[provider_name] = f"Error: {str(e)}"
        
        results_log.append(row_results)
        
        # Optional: save progress every 10 iterations
        if (idx + 1) % 10 == 0:
            progress_df = pd.DataFrame(results_log)
            progress_df.to_csv(f'generation_progress_{DATE_SUFFIX}.csv', index=False)
            print(f"\n  💾 Progress saved after {idx + 1} instructions")
    
    # Save final results
    final_df = pd.DataFrame(results_log)
    final_path = f'generation_results_{DATE_SUFFIX}.csv'
    final_df.to_csv(final_path, index=False)
    
    print("\n" + "=" * 70)
    print(f"✓ All processing complete!")
    print(f"  Results saved to: {final_path}")
    
    return final_df

# Configure which providers to use
providers_to_use = {
    'openai': (generate_openai_image, OPENAI_API_KEY),
    'google_imagen_4_ultra': (generate_google_imagen_4_ultra, GOOGLE_API_KEY),
    'google_imagen_4': (generate_google_imagen_4, GOOGLE_API_KEY),
    'google_imagen_3': (generate_google_imagen_3, GOOGLE_API_KEY),
    'flux_pro_1.1': (generate_flux_pro_image, BFL_API_KEY),
    'flux_pro_1.1_ultra': (generate_flux_pro_ultra_image, BFL_API_KEY),
    'flux_kontext_pro': (generate_flux_kontext_pro_image, BFL_API_KEY),
    'flux_kontext_max': (generate_flux_kontext_max_image, BFL_API_KEY),
    'recraft': (generate_recraft_image, RECRAFT_API_KEY),
    'ideogram': (generate_ideogram_image, IDEOGRAM_API_KEY),
    'fal': (generate_fal_image, FAL_API_KEY),
}

# Run batch processing
results_df = process_all_instructions(lewitt_df, providers_to_use)

print("✓ Batch processing function defined")
print("\nTo process all instructions, uncomment and run the example code above.")